# Loading data

In [69]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sadmansakib7/ecg-arrhythmia-classification-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/ecg-arrhythmia-classification-dataset


In [70]:
import pandas as pd
import numpy as np


import torch


from sklearn.preprocessing import StandardScaler

# Data Preprocessing

In [71]:
data = pd.read_csv("/kaggle/input/ecg-arrhythmia-classification-dataset/INCART 2-lead Arrhythmia Database.csv")
data


,record,type,0_pre-RR,0_post-RR,0_pPeak,0_tPeak,0_rPeak,0_sPeak,0_qPeak,0_qrs_interval,...,1_qPeak,1_qrs_interval,1_pq_interval,1_qt_interval,1_st_interval,1_qrs_morph0,1_qrs_morph1,1_qrs_morph2,1_qrs_morph3,1_qrs_morph4
0,I01,N,163,165,0.069610,-0.083281,0.614133,-0.392761,0.047159,15,...,-0.023370,14,3,23,6,-0.023370,-0.011650,0.082608,0.101373,-0.183387
1,I01,N,165,166,-0.097030,0.597254,-0.078704,-0.078704,-0.137781,3,...,0.081637,15,5,27,7,0.081637,0.102992,0.191225,0.217544,-0.068248
2,I01,N,166,102,0.109399,0.680528,-0.010649,-0.010649,-0.720620,6,...,-0.148539,33,13,52,6,-0.148539,-0.060620,0.081080,0.204400,0.335172
3,I01,VEB,102,231,0.176376,0.256431,-0.101098,-0.707525,-0.101098,4,...,0.046898,21,9,34,4,0.046898,0.083728,0.279512,0.526785,0.450969
4,I01,N,231,165,0.585577,0.607461,-0.083499,-0.083499,-0.167858,3,...,-0.112552,32,5,43,6,-0.112552,0.012989,0.091491,0.134004,0.265232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175724,I75,VEB,205,254,0.167375,0.998450,-1.884460,-2.278752,-1.884460,3,...,-0.063313,31,8,44,5,-0.063313,0.026827,0.679694,1.798788,1.291884
175725,I75,N,254,231,0.067664,0.986674,-0.321804,-0.321804,-0.743924,2,...,-0.038867,5,3,24,16,-0.038867,-0.370870,-0.697097,-0.948059,-1.078708
175726,I75,N,231,229,-0.129754,0.746366,-0.303551,-0.303551,-0.875649,3,...,-0.113876,4,4,24,16,-0.113876,-0.113876,-0.452218,-0.738737,-0.917931
175727,I75,N,229,232,-0.133953,0.747789,-0.626746,-0.626746,-0.871080,2,...,0.087784,18,8,42,16,0.087784,0.107684,0.308452,0.572327,-0.093598


In [72]:
print(data.shape)

(175729, 34)


In [73]:
data.groupby("record")["type"].count()

record
I01    2755
I02    2672
I03    2450
I04    2422
I05    1774
       ... 
I71    1669
I72    2268
I73    1990
I74    2403
I75    2101
Name: type, Length: 75, dtype: int64

In [74]:
data["type"].value_counts()

type
N       153546
VEB      20000
SVEB      1958
F          219
Q            6
Name: count, dtype: int64

In [75]:
# replacing VEB, SVEB, F, Q with general class

def normal_or_not(ex):
    if ex["type"] == "N":
        ex["type"] = 0

    else:
        ex['type'] = 1
    return ex

data = data.apply(normal_or_not, axis=1)

data = data.drop("record", axis=1)
data

,type,0_pre-RR,0_post-RR,0_pPeak,0_tPeak,0_rPeak,0_sPeak,0_qPeak,0_qrs_interval,0_pq_interval,...,1_qPeak,1_qrs_interval,1_pq_interval,1_qt_interval,1_st_interval,1_qrs_morph0,1_qrs_morph1,1_qrs_morph2,1_qrs_morph3,1_qrs_morph4
0,0,163,165,0.069610,-0.083281,0.614133,-0.392761,0.047159,15,2,...,-0.023370,14,3,23,6,-0.023370,-0.011650,0.082608,0.101373,-0.183387
1,0,165,166,-0.097030,0.597254,-0.078704,-0.078704,-0.137781,3,5,...,0.081637,15,5,27,7,0.081637,0.102992,0.191225,0.217544,-0.068248
2,0,166,102,0.109399,0.680528,-0.010649,-0.010649,-0.720620,6,25,...,-0.148539,33,13,52,6,-0.148539,-0.060620,0.081080,0.204400,0.335172
3,1,102,231,0.176376,0.256431,-0.101098,-0.707525,-0.101098,4,3,...,0.046898,21,9,34,4,0.046898,0.083728,0.279512,0.526785,0.450969
4,0,231,165,0.585577,0.607461,-0.083499,-0.083499,-0.167858,3,34,...,-0.112552,32,5,43,6,-0.112552,0.012989,0.091491,0.134004,0.265232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175724,1,205,254,0.167375,0.998450,-1.884460,-2.278752,-1.884460,3,18,...,-0.063313,31,8,44,5,-0.063313,0.026827,0.679694,1.798788,1.291884
175725,0,254,231,0.067664,0.986674,-0.321804,-0.321804,-0.743924,2,10,...,-0.038867,5,3,24,16,-0.038867,-0.370870,-0.697097,-0.948059,-1.078708
175726,0,231,229,-0.129754,0.746366,-0.303551,-0.303551,-0.875649,3,8,...,-0.113876,4,4,24,16,-0.113876,-0.113876,-0.452218,-0.738737,-0.917931
175727,0,229,232,-0.133953,0.747789,-0.626746,-0.626746,-0.871080,2,10,...,0.087784,18,8,42,16,0.087784,0.107684,0.308452,0.572327,-0.093598


In [77]:
typ = data["type"].to_numpy()
typ

array([0, 0, 0, ..., 0, 0, 0])

In [78]:
sscaler = StandardScaler()
features = sscaler.fit_transform(data.drop("type", axis=1))

features

array([[-0.55393466, -0.52165771,  0.15905283, ...,  0.98283272,
         0.98594293,  0.72866882],
       [-0.52152196, -0.50545587, -0.70814796, ...,  1.24883726,
         1.22063578,  0.96419123],
       [-0.50531561, -1.54237313,  0.36611556, ...,  0.97909108,
         1.19408189,  1.78940353],
       ...,
       [ 0.54809728,  0.51525955, -0.87844323, ..., -0.32696011,
        -0.71128087, -0.77387298],
       [ 0.51568458,  0.56386504, -0.90029476, ...,  1.53592747,
         1.93738333,  0.91233524],
       [ 0.56430363,  0.25603023, -1.02407779, ..., -0.35033574,
        -0.71443075, -0.75300624]])